Importación de librerias utiles

In [36]:
import pandas as pd
from sklearn.feature_extraction import _stop_words
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

Definición del clasificador de regresión logistica

In [50]:
Clasificador=LogisticRegression(multi_class='multinomial',  max_iter=10)


Proceso de lectura y filtrado del dataset

In [38]:
#STOP WORDS DE SKLEARN
stop_words=_stop_words.ENGLISH_STOP_WORDS
#LECTURA DEL DATASET
rows_data = pd.read_csv("dataset_final.csv", encoding="latin-1")
#STOP WORDS PERSONALIZADO
rows_stop_words= pd.read_csv("stopwords", sep='\n', header=None)
#OBTENEMOS EL VECTOR DE COMENTARIOS
tweets = rows_data['fullText'].values
#OBTENEMOS EL VECTOR DE LAS POLARIDADES
polaridad = rows_data['polaridad'].values
#UNIMOS LAS STOP WORDS DE SKLEARN CON LAS PERSONAIZADAS
stop_words=stop_words.union(list(rows_stop_words[0]))

Dividimos la data de entrenamiento y la que será usada para test

In [39]:
Tweet_train, Tweet_test, polaridad_train, polaridad_test = train_test_split(tweets, polaridad, test_size=0.1, random_state=4)

Se crea el tokenizador que recibirá el vector de comentarios y convertirá cada comentario en tokens compuesto de 1 o 2 palabras por el hiper-parametro ngram_range=(1,2)

In [40]:
vectorizer = CountVectorizer(analyzer='word', encoding="latin-1", lowercase=True, token_pattern=r'\b\w+\b', max_df=0.9, min_df=0.01, ngram_range=(1,2), stop_words=stop_words)

Se realiza la transformación binaria para posteriormente ajustar el modelo con el vector de comentarios y el vector de polaridades.

In [51]:
vectorizer.fit(Tweet_train)
v_train = vectorizer.transform(Tweet_train)
v_test = vectorizer.transform(Tweet_test)

Clasificador.fit(v_train, polaridad_train)
eficiencia = Clasificador.score(v_test, polaridad_test)

#ES POSIBLE QUE SAGA UN WARNING POR EL LIMITE DE ITERACIONES PERMITIDO QUE DEBERIAS SER MAX 100,
#PERO POR ALGUNA RAZÓN ESTA VERSIÓN DEL SKLEARN ARROJA UN WARNING

#IGNORARL EL WARNING Y SI SEGUIR AL PASO POSTERIOR

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Se hace una predicción y se imprime un reporte de eficiencia del modelo entrenado.

In [42]:
resultado = Clasificador.predict(v_test)
print(classification_report(polaridad_test,resultado ))

              precision    recall  f1-score   support

          -1       0.75      0.75      0.75       115
           0       0.73      0.76      0.74       113
           1       0.86      0.81      0.83        78

    accuracy                           0.77       306
   macro avg       0.78      0.77      0.78       306
weighted avg       0.77      0.77      0.77       306



Ejemplos de comentarios analizados

In [44]:
tweet_negativo = 'lasso corrupto'
tweet_v = vectorizer.transform([tweet_negativo])
respuesta = Clasificador.predict(tweet_v);
print('TWEET NEGATIVO: ', tweet_negativo)
print('POLARIDAD: ', respuesta[0], '\n')

tweet_neutro = 'lasso no hiciste nada'
tweet_v = vectorizer.transform([tweet_neutro])
respuesta = Clasificador.predict(tweet_v);
print('TWEET NEUTRO: ', tweet_neutro)
print('POLARIDAD: ', respuesta[0], '\n')

tweet_positivo = 'lasso gracias por la reactivacion economica'
tweet_v = vectorizer.transform([tweet_positivo])
respuesta = Clasificador.predict(tweet_v);
print('TWEET NEGATIVO: ', tweet_positivo)
print('POLARIDAD: ', respuesta[0], '\n')


TWEET NEGATIVO:  lasso corrupto
POLARIDAD:  -1 

TWEET NEUTRO:  lasso no hiciste nada
POLARIDAD:  0 

TWEET NEGATIVO:  lasso gracias por la reactivacion economica
POLARIDAD:  1 

